In [2]:
import scripts.notebook_utils as utils
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'ticks')
import importlib
import warnings
import os
import pickle
from io import StringIO
warnings.filterwarnings("ignore")

In [3]:
paths = {}
try:
    snakemake
    paths = {k:v for k,v in snakemake.input.items()}
    with open(snakemake.output.paths, 'wb') as f:
        pickle.dump(paths, f)
except NameError:
    with open('./paths.pkl','rb') as f:
        paths = pickle.load(f)

# Load Data

In [4]:
paths

In [5]:
utils = importlib.reload(utils)

In [6]:
raw_data = utils.read_sparse_countmatrix(paths['barcodes'], paths['peaks'], paths['counts'])
corrected_data = utils.read_sparse_countmatrix(paths['barcodes'], paths['peaks'], paths['corrected_counts'])
dupcounts = pd.read_csv(paths['dupcounts'], sep = '\t')
sample_fragments = utils.read_bias_file(paths['sample_fragments'])

In [7]:
raw_data_lsi_model, resolution = utils.process_counts(raw_data, tune = True)

corrected_data_lsi_model, _ = utils.process_counts(corrected_data, 10**resolution)

# Fragment Model Performance

In [8]:
points = utils.benchmark_fragment_model(sample_fragments, dupcounts, raw_data.var)

ax = sns.displot(points, x = 'log_duprate', y = 'true_log_duprate', kind = 'kde')
ax.set(xlabel = 'Predicted Fragment Count', ylabel = 'Observed Fragment Count')

# Compare UMAPs

In [9]:
hue_order = sorted(np.unique(raw_data.obs.leiden))

fig, ax = plt.subplots(1,3, figsize = (14,5))
utils.plot_umap(corrected_data, color_key=raw_data.obs.leiden, quantitative=False, ax = ax[1], hue_order=hue_order)
ax[1].set(title = 'Bias-corrected UMAP', xlabel = 'Colored by UNcorrected clustering')
utils.plot_umap(raw_data, color_key='leiden', quantitative=False, ax = ax[2], legend = True, hue_order=hue_order)
ax[2].set(title = 'Uncorrected UMAP',xlabel = 'Colored by UNcorrected clustering')
utils.plot_umap(corrected_data, color_key=corrected_data.obs.leiden, quantitative=False, ax = ax[0], legend =False)
ax[0].set(title = 'Bias-corrected UMAP', xlabel = 'Colored by corrected cluster')

# Observe Bias Concentration

In [10]:
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax = sns.scatterplot(data = raw_data.obs, x = 'fragment_count', y ='mean_log_duprate', hue = 'leiden', legend = False,
                    hue_order= hue_order, linewidth = 0, s = 10, alpha = 0.8, ax = ax)
ax.set(xscale = 'log', xlabel = 'log(Fragment Count)', ylabel = 'Mean log(duplication rate) per cell')
sns.despine()

In [11]:
fig, ax = plt.subplots(1,2, figsize=(14,5))

utils.plot_umap(raw_data, color_key = 'mean_log_duprate', ax = ax[0])
ax[0].set(title = 'Mean Bias Per Cell')
utils.plot_umap(raw_data, color_key='fragment_count', ax = ax[1])
ax[1].set(title = 'Fragment Count Per Cell')

# Enrichment of Biased Peaks Per Cluster

In [12]:
cluster_enrichments = utils.get_bias_peak_enrichment(raw_data, 'leiden')

sns.displot(data = cluster_enrichments, hue = 'cluster_id', x = 'rank', 
            kind = 'kde', common_norm = False, hue_order= hue_order, height = 7, aspect = 1.5)
plt.show()

# Cell-Specific Effects Analysis

In [13]:
unbiased_rank, biased_rank = np.quantile(raw_data.var['rank'].dropna().values, [0.3,0.7])
peaks_of_interest = pd.concat([
    raw_data.var[raw_data.var['rank'] >= biased_rank].sample(10),
    raw_data.var[raw_data.var['rank'] <= unbiased_rank].sample(10)
])['peak_id'].values

peaks_str = '\n'.join(
    peak.replace('_','\t') for peak in peaks_of_interest
)

In [14]:
fragment_file = paths['fragments']

In [ ]:
relavant_peaks = !bedtools intersect -a $fragment_file -b stdin -sorted < <(echo "$peaks_str" | sort -k1,1 -k2,2n)

In [ ]:
stratified_sample = utils.get_fragment_distribution_by_peak_and_cluster(raw_data, '\n'.join(relavant_peaks), 'leiden')

fig, ax = plt.subplots(figsize = (15,5))
sns.swarmplot(data = stratified_sample.sort_values('rank'), x = 'peak_id', y = 'log_duprate', hue = 'leiden', 
              ax = ax, dodge = True, size = 1.5, 
              order = stratified_sample.groupby('peak_id')['rank'].first().sort_values().index.values)
sns.despine()
ax.set(xticks = [], ylabel = 'Log(Duplication Rate)', xlabel = 'less bias ← Peaks → more bias')
plt.show()

# Differential Peaks Analysis

In [ ]:
diffpeaks = utils.get_differential_peaks(raw_data)

In [ ]:
for cluster, data in diffpeaks.T.iterrows():
    with open(os.path.join(snakemake.output.diffpeaks, 'cluster_{}_diffpeaks.bed'.format(str(cluster))), 'w') as f:
        print('\n'.join([x.replace('_','\t') for x in data.values]), file = f)

# Peak-Bias Heterogeneity

In [ ]:
results = []
for i, l in enumerate(raw_data.var.dropna().sample(20).samples.values):
    for x in l.strip('[]').split(', '):
        results.append((i, float(x)))

peak_samples = pd.DataFrame(results, columns = ['peak_id','log_duprate'])

In [ ]:
fig, ax = plt.subplots(figsize = (15,5))
sns.violinplot(data = peak_samples, x = 'peak_id', y ='log_duprate', ax = ax)
ax.set(xticks = [], ylabel = 'Log(duprate)', xlabel = 'Peaks')
sns.despine()
plt.show()

# Which cells change neighbors?

In [ ]:
utils.compute_neighborhood_change(control=raw_data, treatment=corrected_data, cluster_key='leiden')

In [ ]:
ax = utils.plot_umap(raw_data, color_key='neighbor_change')
ax.set(title = 'Change in neighbors in debiasing treatment')